### Imports

In [10]:
import json
import time

from pyod.models.ecod import ECOD
from tqdm import tqdm
import networkx as nx
import numpy as np
import pandas as pd
import pingouin as pg
import torch
import torch_geometric.transforms as T

from utils.utils_go import *
# from dgi.utils_dgi import *
# from vgae.utils_vgae import *
# from vgae.utils_vgae_tg import *
from unsupervised_models.models import *

# os.environ["DGLBACKEND"] = "pytorch"
# %load_ext autotime

In [11]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [12]:
import torch

!python -c "import torch; print(torch.version.cuda)"
!python -c "import torch; print(torch.__version__)"

11.8
2.0.0


### Parameters

In [13]:
file = open("exp.json")
experiment = json.load(file)
exp_num = experiment["exp"]

file = open("output/{}/parameters.json".format(exp_num))
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

methods = params["methods"]
print("Methods:\t", methods)

data_variations = params["data_variations"]
print("Data variations:", data_variations)

dimension = params["dimension"]
print("Dimension:\t", dimension)

threshold_corr = params["threshold_corr"]
print("Threshold corr:\t", threshold_corr)

iterations = params["iterations"]
print("Iterations:\t", iterations)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id_ = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id_)

seeds = params["seeds"]
print("Seeds:\t\t", seeds)

Exp:		 exp65
Methods:	 ['dgi-tran', 'argva-base', 'vgae-line', 'vgae-base']
Data variations: ['none', 'str', 'dyn']
Dimension:	 3
Threshold corr:	 0.3
Iterations:	 1
Groups id:	 ['WT-YNB', 'S51A-YNB', 'WT-Tyr', 'S51A-Tyr', 'WT-total', 'WT-vac', 'avt1-total', 'avt1-vac', 'aa-standard']
Subgroups id:	 {'WT-YNB': ['1', '2'], 'S51A-YNB': ['1', '2'], 'WT-Tyr': ['1', '2'], 'S51A-Tyr': ['1', '2'], 'WT-total': ['1', '2'], 'WT-vac': ['1', '2'], 'avt1-total': ['1', '2'], 'avt1-vac': ['1', '2'], 'aa-standard': ['1', '2']}
Seeds:		 [41, 42, 43, 44, 45, 46]


### Node-Edge embeddings

In [14]:
# read raw data
df_join_raw = pd.read_csv("input/{}_raw.csv".format(exp), index_col=0)
df_join_raw = df_join_raw.iloc[:, 2:]
df_join_raw

# log10
df_join_raw_log = log10_global(df_join_raw)
df_join_raw_log.head()

epochs = 50 # change
cuda = 0    # change
device = torch.device('cuda:{}'.format(cuda) if torch.cuda.is_available() else 'cpu')
print(device)

runtimes = []

# node-embeddings + edge-embeddings
for method in methods: # change
    for data_variation in data_variations: # change
        print(method, data_variation)
        time_node_embedding = 0
        time_edge_embedding = 0
        for iteration in range(1, iterations + 1):
            # ---
            # Node embeddings
            # ---
            print("Node-embeddings")
            start = time.time()
            subgroups_id = subgroups_id_.copy()
            
            torch.manual_seed(seeds[iteration])
            np.random.seed(seeds[iteration])
            
            if data_variation != "none":
                for group in groups_id:
                    subgroups_id[group] = [data_variation]
            print("Subgroups id:\t", subgroups_id)
            
            for group in groups_id:
                for subgroup in subgroups_id[group]:
                    nodes_data = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup)).iloc[:, 2:]
                    edges_data = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(exp, group, subgroup))

                    if method == "dgi":
                        data = CustomDatasetDGI("g_{}_{}".format(group, subgroup), nodes_data, edges_data)
                        graph = data[0]
                        
                        args_ = args_dgi(dimension)
                        train_dgi(exp, graph, args_, method, group, subgroup, iteration)
                    
                    elif method == "dgi-tran":
                        transform = T.Compose([
                            T.NormalizeFeatures(),
                            T.ToDevice(device),
                            # T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
                        ])
                        name = "{}_{}_{}_{}_{}".format(exp, method, group, subgroup, iteration)
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)
                        model = DGI_Transductive(dataset, dimension, device, name)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))
                    
                    elif method == "dgi-indu":
                        dataset = CustomDataset(nodes_data, edges_data, transform=None)
                        model = DGI_Inductive(dataset, dimension, device)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))
                        
                    elif method == "vgae":
                        data = CustomDatasetVGAE("g_{}_{}".format(group, subgroup), nodes_data, edges_data)
                        graph = data[0]

                        # train
                        args_ = args_vgae(dimension)
                        train_vgae(exp, graph, args_, method, group, subgroup, iteration)
                        
                    elif method == "vgae-base":
                        transform = T.Compose([
                            T.NormalizeFeatures(),
                            T.ToDevice(device),
                            T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
                        ])
                        name = "{}_{}_{}_{}_{}".format(exp, method, group, subgroup, iteration)
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)
                        model = VGAE_Base(dataset, dimension, device, name)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))

                    elif method == "vgae-line":
                        transform = T.Compose([
                            T.NormalizeFeatures(),
                            T.ToDevice(device),
                            T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
                        ])
                        name = "{}_{}_{}_{}_{}".format(exp, method, group, subgroup, iteration)
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)
                        model = VGAE_Linear(dataset, dimension, device, name)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))
                    
                    elif method == "argva-base":
                        transform = T.Compose([
                            T.NormalizeFeatures(),
                            T.ToDevice(device),
                            T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                                            split_labels=True, add_negative_train_samples=False),
                        ])
                        name = "{}_{}_{}_{}_{}".format(exp, method, group, subgroup, iteration)
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)
                        model = ARGVA_Base(dataset, dimension, device, name)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))

                    elif method == "deep-walk":
                        p = 1.0
                        q = 1.0
                        transform = T.Compose([
                            T.ToDevice(device)
                        ])
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)                            
                        model = Node2VecBase(p, q, dataset, dimension, device)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))

                    elif method == "node2vec15":
                        p = 1.0
                        q = 0.5
                        transform = T.Compose([
                            T.ToDevice(device)
                        ])
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)                            
                        model = Node2VecBase(p, q, dataset, dimension, device)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))

                    elif method == "node2vec12":
                        p = 1.0
                        q = 2.0
                        transform = T.Compose([
                            T.ToDevice(device)
                        ])
                        dataset = CustomDataset(nodes_data, edges_data, transform=transform)                            
                        model = Node2VecBase(p, q, dataset, dimension, device)
                        model.fit(epochs=epochs)
                        model.save_node_embeddings("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))
            
            end = time.time()
            elapsed = end - start
            time_node_embedding += elapsed
            
            torch.cuda.empty_cache()
    
            # ---
            # Edge embeddings
            # ---
            print("Edge-embeddings")
            start = time.time()
            subgroups_id = subgroups_id_.copy()
            
            if data_variation != "none":
                subgroups_id_op = {}
                for group in groups_id:
                    subgroups_id_op[group] = [data_variation]
            else:
                subgroups_id_op = subgroups_id
            # print("Subgroups id op:", subgroups_id_op)
            
            edge_embeddings_global(exp, method, groups_id, subgroups_id_op, iteration)
            
            dict_df_edge_embeddings_concat = {}
            for group in tqdm(groups_id):
                df_edge_embeddings_concat = pd.DataFrame()
                k = 0
                for subgroup in tqdm(subgroups_id_op[group]):
                    df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}_{}.csv".format(exp, method, group, subgroup, iteration))
                    df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
                    df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
                    k += 1
                
                # df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(exp, method, group, data_variation, iteration), index=False)
                dict_df_edge_embeddings_concat[group] = df_edge_embeddings_concat
                    
            # ---
            # Outlier detection (ECOD)
            # ---
            print("Outlier detection")
            # dict_df_edge_embeddings_concat_outlier = {}
            dict_df_edge_embeddings_concat_filter = {}

            for group in tqdm(groups_id):
                # df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(exp, method, group, data_variation, iteration))
                df_edge_embeddings_concat = dict_df_edge_embeddings_concat[group]
                
                X_train = df_edge_embeddings_concat.iloc[:, 2:-1]

                clf = ECOD(contamination=0.05, n_jobs=1)
                clf.fit(X_train)

                X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

                df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
                df_edge_embeddings_concat_filter["labels"] = clf.labels_

                # save
                df_edge_embeddings_concat_filter.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_outlier_{}_{}_{}_{}.csv".format(exp, method, group, data_variation, iteration), index=False)
                
                df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0].copy()
                df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

                # dict_df_edge_embeddings_concat_outlier[group] = X_train
                dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter
                
            # mapping idx with id
            for group in tqdm(groups_id):
                df_aux = pd.DataFrame(())
                k = 0
                for subgroup in subgroups_id_op[group]:
                    df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup))
                    dict_id = dict(zip(df_nodes["idx"], df_nodes["id"]))

                    # mapping
                    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
                    df_edge_embeddings_concat_filter_aux = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["subgroup"] == k].copy()
                    
                    # print(df_edge_embeddings_concat_filter)
                    df_edge_embeddings_concat_filter_aux["source"] = df_edge_embeddings_concat_filter_aux["source"].map(dict_id)
                    df_edge_embeddings_concat_filter_aux["target"] = df_edge_embeddings_concat_filter_aux["target"].map(dict_id)
                    df_aux = pd.concat([df_aux, df_edge_embeddings_concat_filter_aux])
                    k += 1
                dict_df_edge_embeddings_concat_filter[group] = df_aux
                
            # format id
            if data_variation != "none":
                for group in tqdm(groups_id):
                    # format
                    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
                    df_edge_embeddings_concat_filter["source"] = df_edge_embeddings_concat_filter["source"].map(lambda x: int(x[1:]))
                    df_edge_embeddings_concat_filter["target"] = df_edge_embeddings_concat_filter["target"].map(lambda x: int(x[1:]))
                        
            # filter by different edges
            if data_variation != "none":
                for group in tqdm(groups_id):
                    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
                    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["source"] != df_edge_embeddings_concat_filter["target"]].copy()
                    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter
                    
            # count edges and filter by count
            dict_df_edges_filter = {}
            for group in tqdm(groups_id):
                # read
                df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
                
                # sort edges
                sort_df_edges(df_edge_embeddings_concat_filter)

                df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["source", "target"]].value_counts().to_frame()
                df_edge_embeddings_concat_filter.reset_index(inplace=True)
                df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]
                
                # filter
                df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
                df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
                dict_df_edges_filter[group] = df_edge_embeddings_concat_filter
                
                df_edge_embeddings_concat_filter.sort_values(["source", "target"], ascending=True, inplace=True)
                df_edge_embeddings_concat_filter.to_csv("output/{}/common_edges/common_edges_{}_{}_{}_{}.csv".format(exp, method, group, data_variation, iteration), index=False)
            
            end = time.time()
            elapsed = end - start
            time_edge_embedding += elapsed
        
        runtimes.append([method, data_variation, time_node_embedding / iterations, time_edge_embedding / iterations])

# save runtimes
df_runtimes = pd.DataFrame(runtimes, columns=["Method", "Data variation", "Runtime node embedding", "Runtime edge embedding"])
df_runtimes.to_csv("output/{}/common_edges/runtimes.csv".format(exp), index=False)

cuda:0
dgi-tran none
Node-embeddings
Subgroups id:	 {'WT-YNB': ['1', '2'], 'S51A-YNB': ['1', '2'], 'WT-Tyr': ['1', '2'], 'S51A-Tyr': ['1', '2'], 'WT-total': ['1', '2'], 'WT-vac': ['1', '2'], 'avt1-total': ['1', '2'], 'avt1-vac': ['1', '2'], 'aa-standard': ['1', '2']}


Epoch: 12:   0%|          | 0/50 [00:00<?, ?it/s, Loss: 1.3869]

Epoch: 18:  34%|███▍      | 17/50 [00:00<00:00, 259.07it/s, Loss: 1.3872]


Early stopping, epoch: 18


Epoch: 8:  14%|█▍        | 7/50 [00:00<00:00, 255.69it/s, Loss: 1.3863]


...

Epoch: 50: 100%|██████████| 50/50 [00:00<00:00, 94.05it/s, Loss: 1.4163]


Edge-embeddings


100%|██████████| 9/9 [00:00<00:00, 112.48it/s]


Outlier detection


100%|██████████| 9/9 [00:00<00:00, 72.35it/s]


In [15]:
# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

In [16]:
# torch.cuda.empty_cache()

In [17]:
# del dataset # data, model, optimizer

In [18]:
# join
list_details = []

for method in methods:
    for k, group in enumerate(groups_id): #
        dict_df_edges_filter = {}
        dict_df_corr = {}
        dict_df_edges_filter_weight = {}
    
        for data_variation in data_variations:
            list_common_subgraph = []
            for iteration in range(1, iterations + 1):
                df_edges_filter_weight_filter = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}_{}.csv".format(exp, method, group, data_variation, iteration))
                # print(df_edges_filter_weight_filter)

                G = nx.from_pandas_edgelist(df_edges_filter_weight_filter) # last change: create_using=nx.Graph() # , edge_attr=["weight"])
                # SG = G.subgraph([0, 1, 2, 3, 4, 5])
                # graph_partial_detail(SG, edges=True)
                list_common_subgraph.append(G)
                
            print("Union")
            # union
            U = nx.compose_all(list_common_subgraph)
            
            df_compose_subgraph = nx.to_pandas_edgelist(U)
            dict_df_edges_filter[group] = df_compose_subgraph.iloc[:, [0, 1]]
            
            # new correlation
            nodes = list(U.nodes())
            # print(len(nodes)) #
            
            df_join_raw_filter = df_join_raw_log.loc[nodes, :]
            # check_dataset(df_join_raw_filter) #
            # print(df_join_raw_filter.describe()) #
            df_join_raw_filter = df_join_raw_filter.filter(like=group, axis=1)

            df_join_raw_filter_t= df_join_raw_filter.T
            check_dataset(df_join_raw_filter_t)
            # df_join_raw_filter_corr = df_join_raw_filter_t.corr(method="pearson")
            df_join_raw_filter_corr = pg.pcorr(df_join_raw_filter_t)
            check_dataset(df_join_raw_filter_corr)
            dict_df_corr[group] = df_join_raw_filter_corr
            
            # get new correlation
            df_edges_filter_weight = dict_df_edges_filter[group].copy()
            df_corr = dict_df_corr[group]

            df_edges_filter_weight["weight"] = df_edges_filter_weight.apply(lambda x: df_corr.loc[x["source"], x["target"]], axis=1)
            df_edges_filter_weight.sort_values(["source", "target"], ascending=True, inplace=True)
            dict_df_edges_filter_weight[group] = df_edges_filter_weight
            
            # common subgraph
            df_edges_filter_weight = dict_df_edges_filter_weight[group]
            # G = nx.from_pandas_edgelist(df_edges_filter_weight, "source", "target", edge_attr="weight")
            print(method, group, data_variation)
            # print("Before")
            # graph_partial_detail(G, edges=True)
                
            # filter by abs(weight) >= threshold
            df_edges_filter_weight = dict_df_edges_filter_weight[group]
            df_edges_filter_weight_filter = df_edges_filter_weight[df_edges_filter_weight["weight"].abs() >= threshold_corr]
            df_edges_filter_weight_filter.to_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, data_variation), index=False)
            
            # print("After")
            # graph_partial_detail(G, edges=True)
            G = nx.from_pandas_edgelist(df_edges_filter_weight_filter, "source", "target", edge_attr="weight")
            list_details.append([method, group, data_variation, G.number_of_nodes(), G.number_of_edges(), nx.density(G)])

df_details = pd.DataFrame(list_details, columns=["Method", "Group", "Data var.", "Num. nodes", "Num. edges", "Density"])
df_details.to_csv("output/{}/common_edges/summary.csv".format(exp), index=False)

Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 440
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 6402
Count zero:	 0
Count positive:	 5698
dgi-tran WT-YNB none
Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 436
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 6266
Count zero:	 0
Count positive:	 5615
dgi-tran WT-YNB str
Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 440
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 6402
Count zero:	 0
Count positive:	 5698


dgi-tran WT-YNB dyn
Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 436
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 5976
Count zero:	 0
Count positive:	 5905
dgi-tran S51A-YNB none
Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 432
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 5878
Count zero:	 0
Count positive:	 5786
dgi-tran S51A-YNB str
Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 436
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 5976
Count zero:	 0
Count positive:	 5905
dgi-tran S51A-YNB dyn
Union
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 436
Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 6092
Count zero:	 0
Count positive:	 5789
dgi-tran WT-Tyr none
Union
Checki